In [1]:
from pymatgen.core import Structure, Lattice
from pymatgen.io.vasp import Poscar
from pymatgen.io.vasp import Vasprun
from pymatgen.io.vasp.outputs import CompleteDos
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.electronic_structure.bandstructure import BandStructureSymmLine
from pymatgen.electronic_structure.dos import Dos
from pymatgen.electronic_structure.plotter import BSPlotter, BSDOSPlotter, BSPlotterProjected, DosPlotter
from pymatgen.electronic_structure.core import Spin
from pymatgen.core.operations import SymmOp
import matplotlib.pyplot as plt
import numpy as np
import nglview as nv
from nglview.shape import Shape

In [63]:
# Define the lattice parameters for hexagonal MoS2
a = 3.19  # Lattice constant 'a' in angstroms
c = 14.879  # Lattice constant 'c' in angstroms

# Define the hexagonal lattice
lattice = Lattice.hexagonal(a, c)

# Atomic basis positions for Mo and S
species = ["Mo", "S", "S"]
frac_coords = [
    [0.000, 0.000, 0.25],  # Mo atom
    [2/3, 1/3, 0.1448],        # S atom
    [2/3, 1/3, 	0.3552],        # S atom
]

# Build the MoS2 structure
mos2_structure = Structure(lattice, species, frac_coords)

In [151]:
print(mos2_structure)
print(mos2_structure.lattice)
#print(mos2_structure.lattice.matrix)

Full Formula (Mo1 S2)
Reduced Formula: MoS2
abc   :   3.190000   3.190000  14.879000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (3)
  #  SP           a         b       c
---  ----  --------  --------  ------
  0  Mo    0         0         0.25
  1  S     0.666667  0.333333  0.1448
  2  S     0.666667  0.333333  0.3552
3.190000 0.000000 0.000000
-1.595000 2.762621 0.000000
0.000000 0.000000 14.879000
[[ 3.19000000e+00  0.00000000e+00  1.95331164e-16]
 [-1.59500000e+00  2.76262104e+00  1.95331164e-16]
 [ 0.00000000e+00  0.00000000e+00  1.48790000e+01]]


In [160]:
# Define the scaling matrix for replication
scaling_matrix = [[3, 0, 0],
                  [0, 3, 0],
                  [0, 0, 1]]

# Create the supercell
supercell = mos2_structure.make_supercell(scaling_matrix, in_place=False)

matrix = supercell.lattice.matrix.copy()

matrix[0] *= 2
matrix[1] *= 2

# Create a new Lattice object
new_lattice = Lattice(matrix)

# Recreate the structure with the new lattice
#supercell = Structure(
#    lattice=new_lattice, 
#    species=supercell.species, 
#    coords=supercell.cart_coords, 
#    coords_are_cartesian=True
#    )

print(supercell)
sites_to_remove = [1,2,5, ]
supercell.remove_sites(sites_to_remove)

# Create a viewer using nglview (for Jupyter Notebooks)
view = nv.show_pymatgen(supercell)
view.clear_representations()
view.add_unitcell()  # Show the unit cell
view.add_representation('ball+stick', color='element')
view

Full Formula (Mo9 S18)
Reduced Formula: MoS2
abc   :   9.570000   9.570000  14.879000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (27)
  #  SP           a         b       c
---  ----  --------  --------  ------
  0  Mo    0         0         0.25
  1  Mo    0         0.333333  0.25
  2  Mo    0         0.666667  0.25
  3  Mo    0.333333  0         0.25
  4  Mo    0.333333  0.333333  0.25
  5  Mo    0.333333  0.666667  0.25
  6  Mo    0.666667  0         0.25
  7  Mo    0.666667  0.333333  0.25
  8  Mo    0.666667  0.666667  0.25
  9  S     0.222222  0.111111  0.1448
 10  S     0.222222  0.444444  0.1448
 11  S     0.222222  0.777778  0.1448
 12  S     0.555556  0.111111  0.1448
 13  S     0.555556  0.444444  0.1448
 14  S     0.555556  0.777778  0.1448
 15  S     0.888889  0.111111  0.1448
 16  S     0.888889  0.444444  0.1448
 17  S     0.888889  0.777778  0.1448
 18  S     0.222222  0.111111  0.3552
 19  S     0.222222  0.444444  0.3552
 20

NGLWidget()

In [137]:
# Define the reflection plane (60 degrees)
#reflection_matrix = np.array([[-0.5, np.sqrt(3)/2, 0], [np.sqrt(3)/2, 0.5, 0], [0, 0, 1]])
#reflection_matrix = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])
reflection_matrix = np.array([[-0.5, np.sqrt(3)/2, 0], [np.sqrt(3)/2, 0.5, 0], [0, 0, 1]])

translation_vector = np.array([0, 1/2, 0])
translation_vector = translation_vector[0]*matrix[0] + translation_vector[1]*matrix[1] + translation_vector[2]*matrix[2]

# Define the reflection operation with translation
reflection_op = SymmOp.from_rotation_and_translation(
    rotation_matrix=reflection_matrix,
    translation_vec=translation_vector
)

# Apply reflection operation
reflected_structure = supercell.copy()
reflected_structure.apply_operation(reflection_op)

# Combine the original and reflected structures
combined_structure = supercell.copy()
for site in reflected_structure:
    combined_structure.append(site.species, site.coords, coords_are_cartesian=True)

translation_vector = np.array([0, 0, 0])
translation_vector = translation_vector[0]*matrix[0] + translation_vector[1]*matrix[1] + translation_vector[2]*matrix[2]

reflection_op = SymmOp.from_rotation_and_translation(
    translation_vec=translation_vector
)
combined_structure.apply_operation(reflection_op)

# Create a viewer using nglview (for Jupyter Notebooks)
view = nv.show_pymatgen(combined_structure)
view.clear_representations()
view.add_unitcell()  # Show the unit cell
view.add_representation('ball+stick', color='element')
view

NGLWidget()

In [107]:
# Define the scaling matrix for replication
scaling_matrix = [[2, 0, 0],
                  [0, 1, 0],
                  [0, 0, 1]]

# Create the supercell
combined_structure.make_supercell(scaling_matrix, in_place=True)

# Create a viewer using nglview (for Jupyter Notebooks)`
view = nv.show_pymatgen(combined_structure)
view.clear_representations()
view.add_unitcell()  # Show the unit cell
view.add_representation('ball+stick', color='element')
view

NGLWidget()

In [47]:
# Define the scaling matrix for replication
scaling_matrix = [[1, 0, 0],
                  [0, 1, 0],
                  [0, 0, 1]]

# Create the supercell
combined_super_structure = combined_structure.make_supercell(scaling_matrix, in_place=False)

# Create a viewer using nglview (for Jupyter Notebooks)
view = nv.show_pymatgen(combined_super_structure)
view.clear_representations()
view.add_unitcell()  # Show the unit cell
view.add_representation('ball+stick', color='element')
view

NGLWidget()

In [ ]:
print()